## Лабораторная работа №5
### Задача: 
### $ \left\{\begin{matrix}
\frac{\partial^2 u}{\partial x^2} +\frac{\partial^2 u}{\partial y^2} = -2(x(1-x) + y(1-y)), \\ 
G = {(x,y):0\leq x, y\leq 1};\\ 
u|_{\partial G} = 0;
\end{matrix}\right. $
    
### Аналитическое решение: 
### $ u = x(1-x)y(1-y) $
    
### Разностная схема: 
### $ \left\{\begin{matrix}
\frac{u^{n+1}_{m,l} - u^{n}_{m,l}}{\tau} = \frac{u^{n}_{m-1,l} - 2u^{n}_{m,l} + u^{n}_{m+1,l}}{h_{x}^{2}} + \frac{u^{n}_{m,l-1} - 2u^{n}_{m,l} + u^{n}_{m,l+1}}{h_{y}^{2}} + 2(x_{m}(1-x_{m}) + y_{l}(1-y_{l})), \\ 
l = \overline{1, L-1}, m = \overline{1, M-1}, n = \overline{1, N};\\ 
u_{m,l}^{0} = \psi_{m,l}, l = \overline{0, L}, m = \overline{0, M};\\ 
u^{n}_{m,0} = u^{n}_{m,L} = 0, m = \overline{0, M}, n = \overline{1, N};\\
u^{n}_{0,l} = u^{n}_{M,l} = 0, l = \overline{1, L-1}, n = \overline{1, N};\\
\end{matrix}\right. $
    

In [44]:
import numpy as np
import pandas as ps
import math

def p(array):  # вывод таблиц
    return ps.DataFrame(array)

def analytical_solution(x, y):   # вывод аналитического решения
    array = np.zeros((len(y), len(x)))
    for m in range(len(x)):
        for l in range(len(y)):
            array[l][m] = x[m]*(1-x[m])*y[l]*(1-y[l])
    return array

def succession(N):
    succession = [N]
    a = N
    while a > 1:
        if a%2 == 0:
            a /= 2
            succession.append(a)
        if a%2 == 1:
            a -= 1
            succession.append(a)
    succession.remove(0)
    succession.reverse()
    return succession

def tau_formula(tetta, N, max_mu, min_mu):
    tau = 2/((max_mu + min_mu) + (max_mu - min_mu)*np.cos(np.pi*(2*tetta - 1)/(2*N)))
    return tau

def tau_sequence(N, max_mu, min_mu):
    sequence = succession(N) # {1, 2, 4, 5, ..., N}
    tetta = np.zeros((len(sequence), sequence[len(sequence) - 1] + 1))
    tetta[0][1] = 1
    for n in sequence[1:]:
        i = sequence.index(n)

        if n % 2 == 0:
            m = int(n/2)

            if sequence[i + 1] % 2 == 0: 

                for j in range(1, m + 1):
                    tetta[i][2*j - 1] = tetta[i - 1][j]
                    tetta[i][2*j] = 4*m - tetta[i][2*j - 1]     
            else:

                for j in range(1, m + 1):
                    tetta[i][2*j - 1] = tetta[i - 1][j]
                    tetta[i][2*j] = 4*m + 2 - tetta[i][2*j - 1]   

        if n % 2 == 1:
            m = int((n - 1)/2)

            for j in range(1, 2*m + 1):
                tetta[i][j] = tetta[i - 1][j]
            tetta[i][2*m + 1] = 2*m + 1
            
    tetta = tetta[tetta.shape[0] - 1:]
    tetta = np.delete(tetta, 0)
    tetta = (tetta + 1)/2
    tau_sequence = tau_formula(tetta, N, max_mu, min_mu)
    return tau_sequence

x0, y0 = 0, 0
xM, yL = 1, 1
h = 0.1
x = np.linspace(x0, xM, 10 + 1)
y = np.linspace(y0, yL, 10 + 1)

def Chebyshev_method(N):   # метод Чебышёва
    
    M = int((xM - x0)/h)
    L = int((yL - y0)/h)
    x = np.linspace(x0, xM, L + 1)
    y = np.linspace(y0, yL, M + 1)
    max_mu = 4*(L**2 + M**2)
    min_mu = 2*np.pi**2
    
    u = np.zeros((N+1,M + 1, L + 1))  # начальное приближение, удвл ГУ
    tau = tau_sequence(N, max_mu, min_mu)
    for n in range(N):
        for m in range(1, M):
            for l in range(1, L):
                u[n+1][l][m] = u[n][l][m] + (u[n][l][m+1] - 4*u[n][l][m] + u[n][l][m-1] + u[n][l+1][m] + u[n][l-1][m])*tau[n]/h**2 + 2*tau[n]*(x[m]*(1 - x[m]) + y[l]*(1 - y[l]))
    
    u = u[N][:][:]
    rate = np.fabs(u - analytical_solution(x, y))
    
    df_u = ps.DataFrame(u, x, x)
    df_u = df_u.iloc[::int(0.2/h), ::int(0.2/h)]
    df_an = ps.DataFrame(analytical_solution(x, y), x, x)
    df_an = df_an.iloc[::int(0.2/h), ::int(0.2/h)]
    df_rate = ps.DataFrame(rate, x, x)   
    df_rate = df_rate.iloc[::int(0.2/h), ::int(0.2/h)]
    print('N = ' + str(N))
    
    
    return df_an, df_u, df_rate# analytical_solution(x, y), u, rate

an_solution = Chebyshev_method(3)[0]#|.iloc[::2, ::2]

def df(array):
    x = np.arange(0, 1 + h, h)
    df_array = ps.DataFrame(array, x, x)
    df_array = df_array.iloc[::int(0.2/h), ::int(0.2/h)]
    
    return df_array

N = 3


### Ответ (при M = L = 10, h = 0.1):

Аналитическое решение:

In [45]:
an_solution

,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0
0.2,0.0,0.0256,0.0384,0.0384,0.0256,0.0
0.4,0.0,0.0384,0.0576,0.0576,0.0384,0.0
0.6,0.0,0.0384,0.0576,0.0576,0.0384,0.0
0.8,0.0,0.0256,0.0384,0.0384,0.0256,0.0
1.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0


N = 5

Решение:

In [46]:
Chebyshev_method(5)[1]

N = 5


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0.2,0.0,0.017604,0.024164,0.024164,0.017604,0.0
0.4,0.0,0.024164,0.032509,0.032509,0.024164,0.0
0.6,0.0,0.024164,0.032509,0.032509,0.024164,0.0
0.8,0.0,0.017604,0.024164,0.024164,0.017604,0.0
1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


Погрешность:

In [47]:
Chebyshev_method(5)[2]

N = 5


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0.2,0.0,0.007996,0.014236,0.014236,0.007996,0.0
0.4,0.0,0.014236,0.025091,0.025091,0.014236,0.0
0.6,0.0,0.014236,0.025091,0.025091,0.014236,0.0
0.8,0.0,0.007996,0.014236,0.014236,0.007996,0.0
1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


N = 11

Решение:

In [48]:
Chebyshev_method(11)[1]

N = 11


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0.2,0.0,0.023970,0.035924,0.035924,0.023970,0.0
0.4,0.0,0.035924,0.053878,0.053878,0.035924,0.0
0.6,0.0,0.035924,0.053878,0.053878,0.035924,0.0
0.8,0.0,0.023970,0.035924,0.035924,0.023970,0.0
1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


Погрешность:

In [49]:
Chebyshev_method(11)[2]

N = 11


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0.2,0.0,0.001630,0.002476,0.002476,0.001630,0.0
0.4,0.0,0.002476,0.003722,0.003722,0.002476,0.0
0.6,0.0,0.002476,0.003722,0.003722,0.002476,0.0
0.8,0.0,0.001630,0.002476,0.002476,0.001630,0.0
1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


N = 21

Решение:

In [50]:
Chebyshev_method(21)[1]

N = 21


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0.2,0.0,0.025522,0.038283,0.038283,0.025522,0.0
0.4,0.0,0.038283,0.057422,0.057422,0.038283,0.0
0.6,0.0,0.038283,0.057422,0.057422,0.038283,0.0
0.8,0.0,0.025522,0.038283,0.038283,0.025522,0.0
1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


Погрешность:

In [51]:
Chebyshev_method(21)[2]

N = 21


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0.2,0.0,0.000078,0.000117,0.000117,0.000078,0.0
0.4,0.0,0.000117,0.000178,0.000178,0.000117,0.0
0.6,0.0,0.000117,0.000178,0.000178,0.000117,0.0
0.8,0.0,0.000078,0.000117,0.000117,0.000078,0.0
1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


N = 41

Решение:

In [52]:
Chebyshev_method(41)[1]

N = 41


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0
0.2,0.0,0.0256,0.0384,0.0384,0.0256,0.0
0.4,0.0,0.0384,0.0576,0.0576,0.0384,0.0
0.6,0.0,0.0384,0.0576,0.0576,0.0384,0.0
0.8,0.0,0.0256,0.0384,0.0384,0.0256,0.0
1.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0


Погрешность:

In [53]:
Chebyshev_method(41)[2]

N = 41


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
0.2,0.0,1.990881e-07,3.089128e-07,3.089128e-07,1.990881e-07,0.0
0.4,0.0,3.089128e-07,4.790213e-07,4.790213e-07,3.089128e-07,0.0
0.6,0.0,3.089128e-07,4.790213e-07,4.790213e-07,3.089128e-07,0.0
0.8,0.0,1.990881e-07,3.089128e-07,3.089128e-07,1.990881e-07,0.0
1.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0


N = 81

Решение:

In [54]:
Chebyshev_method(81)[1]

N = 81


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0
0.2,0.0,0.0256,0.0384,0.0384,0.0256,0.0
0.4,0.0,0.0384,0.0576,0.0576,0.0384,0.0
0.6,0.0,0.0384,0.0576,0.0576,0.0384,0.0
0.8,0.0,0.0256,0.0384,0.0384,0.0256,0.0
1.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0


Погрешность:

In [55]:
Chebyshev_method(81)[2]

N = 81


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
0.2,0.0,1.758944e-12,2.802050e-12,2.802050e-12,1.758933e-12,0.0
0.4,0.0,2.802057e-12,4.456061e-12,4.456054e-12,2.802029e-12,0.0
0.6,0.0,2.802043e-12,4.456040e-12,4.456054e-12,2.802029e-12,0.0
0.8,0.0,1.758930e-12,2.802023e-12,2.802029e-12,1.758930e-12,0.0
1.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0


Максимальная сходимость примерно при N = 191

Решение:

In [84]:
Chebyshev_method(191)[1]

N = 191


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0
0.2,0.0,0.0256,0.0384,0.0384,0.0256,0.0
0.4,0.0,0.0384,0.0576,0.0576,0.0384,0.0
0.6,0.0,0.0384,0.0576,0.0576,0.0384,0.0
0.8,0.0,0.0256,0.0384,0.0384,0.0256,0.0
1.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0


Погрешность:

In [83]:
Chebyshev_method(191)[2]

N = 191


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
0.2,0.0,3.469447e-18,6.938894e-18,0.000000e+00,3.469447e-18,0.0
0.4,0.0,6.938894e-18,0.000000e+00,0.000000e+00,6.938894e-18,0.0
0.6,0.0,1.387779e-17,6.938894e-18,6.938894e-18,0.000000e+00,0.0
0.8,0.0,0.000000e+00,6.938894e-18,0.000000e+00,3.469447e-18,0.0
1.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0


#### Из результатов видно, что ошибка растет при увеличении шага по пространству. Это происходит потому, что увеличивается множитель $ \tau_{n}/h^{2} $ из разностной схемы, тем самым ухудшая ее устойчивость.


In [61]:
def df(array):
    x = np.arange(0, 1 + h, h)
    df_array = ps.DataFrame(array, x, x)
    df_array = df_array.iloc[::int(0.2/h), ::int(0.2/h)]
    
    return df_array



In [59]:
a = np.array([[1,1,1,1,1],[2,2,2,2,2],[3,3,3,3,3],[4,4,4,4,4],[5,5,5,5,5]])
a = p(a)
a

,0,1,2,3,4
0,1,1,1,1,1
1,2,2,2,2,2
2,3,3,3,3,3
3,4,4,4,4,4
4,5,5,5,5,5


In [35]:
a.iloc[::2, ::2]

,0,2,4
0,1,1,1
2,3,3,3
4,5,5,5


In [34]:
0.2/h

2.0